In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
#!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
from tensorflow.keras import layers, models

from  sklearn.model_selection import train_test_split

# 헬퍼(helper) 라이브러리를 임포트합니다
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [0]:
train = pd.read_csv("./drive/My Drive/data/kannada/train.csv")
test  = pd.read_csv("./drive/My Drive/data/kannada/test.csv")

#dig = pd.read_csv("./drive/My Drive/data/kannada/Dig-MNIST.csv")

In [0]:
#train = pd.concat([train, dig])

In [0]:
train_images = train.iloc[:,1:]
train_labels = train["label"]
test_images = test.iloc[:,1:]

In [0]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

In [0]:
class_names = ['0','1','2','3','4','5','6','7','8','9']

In [0]:
x_train, x_valid, x_test = x_train.apply(lambda x: x/255), x_valid.apply(lambda x: x/255), test_images.apply(lambda x: x/255)

In [0]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
x_valid = x_valid.values.reshape(-1, 28, 28, 1)
x_test = x_test.values.reshape(-1, 28, 28, 1)

In [0]:
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

In [0]:
# Learning Rate 조절 필요

model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 128)       7

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs=20)

Train on 59704 samples
Epoch 1/20
59704/59704 [==============================] - 17s 290us/sample - loss: 0.1590 - accuracy: 0.9520
Epoch 2/20
59704/59704 [==============================] - 14s 240us/sample - loss: 0.0703 - accuracy: 0.9792
Epoch 3/20
59704/59704 [==============================] - 14s 238us/sample - loss: 0.0564 - accuracy: 0.9829
Epoch 4/20
59704/59704 [==============================] - 14s 238us/sample - loss: 0.0470 - accuracy: 0.9857
Epoch 5/20
59704/59704 [==============================] - 14s 237us/sample - loss: 0.0389 - accuracy: 0.9879
Epoch 6/20
59704/59704 [==============================] - 14s 236us/sample - loss: 0.0334 - accuracy: 0.9898
Epoch 7/20
59704/59704 [==============================] - 14s 236us/sample - loss: 0.0287 - accuracy: 0.9910
Epoch 8/20
59704/59704 [==============================] - 14s 236us/sample - loss: 0.0259 - accuracy: 0.9919
Epoch 9/20
59704/59704 [==============================] - 14s 238us/sample - loss: 0.0218 - accuracy: 0.9

In [0]:
test_loss, test_acc = model.evaluate(x_valid, y_valid, verbose=2)

10536/1 - 1s - loss: 0.0193 - accuracy: 0.9917


model.fit(x_train, y_train, epochs=20)

6000/1 - 2s - loss: 0.0057 - accuracy: 0.9955

In [0]:
pred = model.predict(x_test)

In [0]:
y_test = np.argmax(pred, axis=1)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

5000/1 - 1s - loss: 0.0126 - accuracy: 1.0000


In [0]:
test['label'] = np.argmax(pred, axis=1)

In [0]:
sub = test[['id', 'label']]

In [0]:
sub.to_csv('submission.csv', index=False)